## Imports

In [27]:
import numpy as np
import linecache as lc
import pandas as pd

# Aggregating postcodes by alphabetical prefix

In [28]:
customer_id_og = lc.getline(r"CaseStudyData.txt", 17).split(" ")[2:-1]
customer_id_og = [id.strip('"') for id in customer_id_og]

candidate_id_og = lc.getline(r"CaseStudyData.txt", 27).split(" ")[2:-1]
candidate_id_og = [id.strip('"') for id in candidate_id_og]

In [29]:
def extract_first_occurrence(postcodes_list):
    first_occurrences = {}
    
    for i, postcode in enumerate(postcodes_list):
        prefix = postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else (postcode[0] if postcode and postcode[0].isalpha() else '')
        # prefix = postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else (postcode[0] if postcode[0].isalpha() else '')
        if prefix not in first_occurrences:
            first_occurrences[prefix] = {'postcode': postcode, 'index': i}
    
    unique_prefixes = list(first_occurrences.keys())
    first_postcodes = [first_occurrences[prefix]['postcode'] for prefix in unique_prefixes]
    first_indices = [first_occurrences[prefix]['index'] for prefix in unique_prefixes]
    
    return unique_prefixes, first_postcodes, first_indices

customer_id_selective = extract_first_occurrence(customer_id_og)
candidate_id_selective = extract_first_occurrence(candidate_id_og)

# Aggregating the 1D vectors

In [30]:
def aggregate_postcodes(list):
    
    # Extract unique prefixes
    unique_prefixes = np.unique([postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else postcode[0] for postcode in list])

    # Create array with prefixes
    aggregated_values = np.array([postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else postcode[0] for postcode in list])

    return unique_prefixes, aggregated_values

In [31]:
def process_aggregated_values(postcode_list, value_list, operation='average'):
    
    '''
    postcode_list: the reference data to obtain the unique postcode prefixes from
    value_list: the vector we wish to aggregate/group
    operation: can choose between averging, summing, or taking the maximum value when aggregating
    
    This function takes the arguments listed above and returns a vector of 
    aggregated values based on the operation chosen.
    '''
    
    unique_prefixes, aggregated_values = aggregate_postcodes(postcode_list)
    
    values_dict = {prefix: [] for prefix in unique_prefixes}

    for prefix, value in zip(aggregated_values, value_list):
        values_dict[prefix].append(value)

    if operation == 'average':
        result_array = np.array([np.mean(values_dict[prefix]) for prefix in unique_prefixes])
    elif operation == 'sum':
        result_array = np.array([np.sum(values_dict[prefix]) for prefix in unique_prefixes])
    elif operation == 'maximum':
        result_array = np.array([np.max(values_dict[prefix]) for prefix in unique_prefixes])
    else:
        raise ValueError("Invalid operation. Please choose 'average', 'sum', or 'maximum'.")

    return result_array

In [32]:
def extract_selected_values(postcode_list, value_list):
    first_indices = extract_first_occurrence(postcode_list)[2]
    return [value_list[i] for i in first_indices]

## Aggregating coordinates

In [33]:
customer_easting = lc.getline(r"CaseStudyData.txt", 20).split(" ")[2:-1]
customer_easting = [eval(coord) for coord in customer_easting]
customer_easting_selective = extract_selected_values(customer_id_og, customer_easting)

customer_northing = lc.getline(r"CaseStudyData.txt", 21).split(" ")[2:-1]
customer_northing = [eval(coord) for coord in customer_northing]
customer_northing_selective = extract_selected_values(customer_id_og, customer_northing)

candidate_easting = lc.getline(r"CaseStudyData.txt", 30).split(" ")[2:-1]
candidate_easting = [eval(coord) for coord in candidate_easting]
candidate_easting_selective = extract_selected_values(candidate_id_og, candidate_easting)

candidate_northing = lc.getline(r"CaseStudyData.txt", 31).split(" ")[2:-1]
candidate_northing = [eval(coord) for coord in candidate_northing]
candidate_northing_selective = extract_selected_values(candidate_id_og, candidate_northing)

## Aggregating  warehouse costs and capacities

In [34]:
def process_costs_capacities(line_start, line_stop):
    
    '''
    Simply takes the line to start reading, and the line to stop reading. Then, it 
    processes and transforms the 1D cost and capacity vectors into a format that we can use.
    '''
    
    file = open(r"CaseStudyData.txt", "r")
    costs_og = file.readlines()[line_start-1:line_stop]
    costs_og = list(map(lambda s: s.strip(), costs_og))

    costs_og = [costs_og[line].split(" ") for line in range(len(costs_og))]
    costs = []
    for line in costs_og:
        for i in range(len(line)):
            costs.append(line[i].strip('[]'))
            
    costs = costs[2:]        
    costs = [eval(cost) for cost in costs]
    return costs

In [35]:
setup_costs = process_costs_capacities(62, 105)
operating_costs = process_costs_capacities(108, 146)
wh_capacities = process_costs_capacities(149, 193)

setup_costs_selective = extract_selected_values(customer_id_og, setup_costs)
operating_costs_selective = extract_selected_values(customer_id_og, operating_costs)
wh_capacities_selective = extract_selected_values(customer_id_og, wh_capacities)

In [36]:
customer_populations = lc.getline(r"CaseStudyData.txt", 24).split(" ")[2:-1]
customer_populations = [eval(coord) for coord in customer_populations]
customer_populations_selective = extract_selected_values(customer_id_og, customer_populations)

# Aggregating the 2D arrays

We start off with 4 $\times$ 440 = 1760 values, so we expect to be left with 4 vectors in the end - one for each product type.

In [37]:
def process_nD(line_start, line_stop, no_to_delete = None):
    
    '''
    Very similar to the function for processing costs and capacities, but this 
    one doesn't automatically remove the first two elements from the beginning like
    the other one. This is because the formatting for 2D stuff is a little different 
    and requires some further processing outside the function too.
    
    The logic is to basically get it all cleaned up within the function, then 
    reshape and delete columns outside.
    '''
    
    file = open(r"CaseStudyData.txt", "r")
    list_og = file.readlines()[line_start-1:line_stop]
    list_og = list(map(lambda s: s.strip(), list_og))

    list_og = [list_og[line].split(" ") for line in range(len(list_og))]
    processed_list = []
    for line in list_og:
        for i in range(len(line)):
            processed_list.append(line[i].strip('[]'))
    
    if no_to_delete != None:
        processed_list = processed_list[no_to_delete:]
    else:
        pass
    return processed_list

In [38]:
demand_per_product = np.asarray(process_nD(197, 343)).reshape(440, 6)[:, 2:]
demand_per_product = demand_per_product.astype(int)

dpp_selective = []
for i in range(4):
    dpp_selective.append(process_aggregated_values(customer_id_og, demand_per_product[:,i], "sum"))

dpp_selective = np.asarray(dpp_selective)

# Aggregating the 3D array

In [39]:
dpp_per_year = np.asarray(process_nD(347, 1693, 1)).reshape(440, 4, -1)[:, :, 3:]
dpp_per_year = dpp_per_year.astype(int)

dpp_py_selective = []
for i in range(4):
    dpp_sel = []
    for j in range(10):
        dpp_sel.append(process_aggregated_values(customer_id_og, dpp_per_year[:, i, j], "sum"))
    dpp_py_selective.append(dpp_sel)
dpp_py_selective = np.asarray(dpp_py_selective)
dpp_py_selective[0].shape

(10, 15)

# Aggregating the 4D array

In [40]:
dppy_scenarios = np.asarray(process_nD(1695, 115666, 1)).reshape(440, 4, 10, -1)[:, :, :, 4:]
dppy_scenarios  = dppy_scenarios.astype(int)

dppys_selective = []
for i in range(4):
    dppy_sel = []
    for j in range(10):
        dp_sel = []
        for k in range(100):
            dp_sel.append(process_aggregated_values(customer_id_og, dppy_scenarios[:, i, j, k], "sum"))
        dppy_sel.append(dp_sel)
    dppys_selective.append(dppy_sel)
    
dppys_selective = np.asarray(dppys_selective)

In [41]:
can_supp_distances = np.asarray(process_nD(115670, 116109)).reshape(440, -1)[:, 2:]
can_supp_distances  = can_supp_distances.astype(float)

indices = extract_first_occurrence(customer_id_og)[2]

agg_dist_can_supp = np.asarray([can_supp_distances[i,:] for i in indices])
pd.DataFrame(agg_dist_can_supp)

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,192.61,175.97,179.64,18.06,141.07,190.36,50.89,179.24,93.43,101.05,...,121.47,65.56,4.55,18.26,67.08,102.44,116.63,86.38,134.37,144.26
1,180.02,112.58,147.99,56.74,77.68,110.06,18.03,115.41,30.04,37.23,...,53.65,3.07,63.82,79.39,128.22,125.29,53.24,23.00,70.55,80.44
2,265.51,131.58,218.67,203.08,105.62,25.49,165.12,95.71,125.63,100.62,...,91.59,146.53,210.16,225.73,251.09,234.72,96.99,126.45,73.15,65.86
3,209.66,101.83,177.63,111.65,65.06,52.95,72.84,54.00,49.65,24.97,...,26.93,61.04,118.74,134.31,171.31,154.93,39.42,44.00,10.32,38.37
4,211.69,73.99,147.48,116.74,44.53,71.06,78.95,81.73,47.88,31.08,...,3.49,60.19,123.82,139.39,173.34,156.96,18.88,48.69,36.87,21.60
5,189.60,55.68,142.76,135.83,29.71,57.91,97.87,99.02,58.39,51.35,...,24.35,79.29,142.92,158.49,183.85,167.47,29.75,59.20,54.15,19.64
6,106.26,269.55,220.07,288.81,277.58,350.65,303.85,375.09,275.42,302.76,...,308.35,284.10,299.77,281.33,231.83,196.74,300.84,271.27,330.23,311.33
7,87.60,147.24,97.76,96.37,166.23,215.52,132.69,216.05,116.38,143.72,...,149.31,125.05,106.18,87.75,38.24,4.79,141.80,112.23,171.19,177.32
8,205.30,71.38,158.47,159.86,45.42,61.03,121.90,119.41,82.42,77.33,...,48.38,103.32,166.95,182.52,207.88,191.50,53.78,83.23,74.55,34.60
9,170.09,247.49,198.01,198.62,268.48,317.78,234.94,318.30,218.63,245.98,...,251.56,227.31,208.43,190.00,140.49,107.04,244.05,214.48,273.44,279.57


In [42]:
can_cust_distances = np.asarray(process_nD(116114, 116553)).reshape(440, -1)[:, 2:]
can_cust_distances  = can_cust_distances.astype(float)

indices = extract_first_occurrence(customer_id_og)[2]

agg_dist_customer_only = np.asarray([can_cust_distances[:,i] for i in indices]).T

agg_dist_both = np.asarray([agg_dist_customer_only[i,:] for i in indices])

pd.DataFrame(agg_dist_customer_only)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.00,64.86,211.20,119.78,124.86,143.95,298.79,105.21,167.98,207.46,92.74,141.36,153.11,83.21,156.20
1,1.53,65.78,212.12,120.69,125.77,144.87,298.73,105.15,168.90,207.40,93.66,142.28,154.03,84.13,157.11
2,3.82,61.98,208.32,116.90,121.98,141.08,305.21,111.62,165.11,213.87,89.87,138.48,150.23,80.33,153.32
3,5.34,63.62,209.96,118.53,123.62,142.71,298.88,106.44,166.74,208.70,91.50,140.12,151.87,81.97,154.95
4,9.12,64.56,210.90,119.48,124.56,143.66,297.83,105.39,167.69,207.64,92.45,141.07,152.82,82.91,155.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,175.35,111.53,85.48,50.12,77.85,95.13,371.20,212.16,115.53,314.42,85.17,84.75,104.14,102.23,27.60
436,172.77,108.95,82.39,49.15,75.27,92.55,368.62,209.58,112.95,311.83,82.59,82.17,101.55,99.65,26.06
437,162.43,98.61,98.41,37.20,64.93,82.21,358.28,199.24,102.61,301.50,72.25,71.83,91.22,89.31,40.52
438,175.05,111.23,99.84,49.82,77.54,94.83,370.90,211.86,115.22,314.11,84.87,84.45,103.83,101.93,41.96


In [43]:
for i, j in enumerate(indices):
    print(agg_dist_both[i, i] == can_cust_distances[j, j])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


# Writing to a txt file

In [46]:
# Strings to be written before each array
# File path
output_file = f"CaseStudyData_Aggregated_Testing.txt"

# Writing to the file
with open(output_file, 'w') as file:
    # Write number of customers, locations
    file.write("! Number of customers = postcode districts\n")
    file.write(f"nbCustomers: {len(customer_id_selective[0])}\n")
    file.write("! Number of candidate locations\n")
    file.write(f"nbCandidates: {len(candidate_id_selective[0])}\n \n")
    
    # Write Customer IDs
    file.write("! Vector of customer ids\n")
    customer_id_output = "CustomerId: [ " + " ".join('"' + s + '"' for s in customer_id_selective[0]) + " ]\n"
    file.write(customer_id_output)

    # Write Customer eastings and northings
    file.write("\n! Vector of customer coordinates\n")
    file.write("CustomerEasting: [" )
    np.savetxt(file, customer_easting_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n")
    file.write("CustomerNorthing: [" )
    np.savetxt(file, customer_northing_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")

    # Write Customer populations
    file.write("! Vector of customer populations\nCustomerPopulation: [ ")
    np.savetxt(file, customer_populations_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")

    # Write Candidate IDs
    file.write("! Vector of candidate location ids\n")
    candidate_id_output = "CandidateId: [ " + " ".join('"' + s + '"' for s in candidate_id_selective[0]) + " ]\n"
    file.write(candidate_id_output)

    # Write Customer eastings and northings
    file.write("\n! Vector of candidate location coordinates\n")
    file.write("CandidateEasting: [" )
    np.savetxt(file, candidate_easting_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n")
    file.write("CandidateNorthing: [" )
    np.savetxt(file, candidate_northing_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")

    # Write set up and operating costs
    file.write("! Setup cost for warehouses\nSetup: [(1) ")
    np.savetxt(file, setup_costs_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")
    file.write("! Operating cost for warehouses\nOperating: [(1) ")
    np.savetxt(file, operating_costs_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")
    file.write("! The warehouse capacity\nCapacity: [(1) ")
    np.savetxt(file, wh_capacities_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")

    # Write annual district demand in kg per product group
    file.write("! The annual district demand in kilograms per product group\nCustomerDemand: [\n")
    for i in range(len(customer_id_selective[0])):
        file.write(f"({i+1} 1) {dpp_selective[0,i]} {dpp_selective[1,i]} {dpp_selective[2,i]} {dpp_selective[3,i]} \n")
    file.write("]\n \n")

    # Write customer demand in kilograms per product group over nbPeriod years
    file.write("! The customer demand in kilograms per product group over nbPeriods years\nCustomerDemandPeriods: [")
    for cust in range(len(customer_id_selective[0])):
        for pro in range(4):
            file.write(f"\n({cust+1} {pro+1} 1) ")
            for per in range(10):
                file.write(f"{dpp_py_selective[pro, per, cust]} ")
    file.write("]\n \n")

    # Write customer demand per period for each scenario
    file.write("CustomerDemandPeriodScenarios: [")
    for cust in range(len(customer_id_selective[0])):
        for pro in range(4):
            for per in range(10):
                file.write(f"\n({cust+1} {pro+1} {per+1} 1) ")
                for scen in range(100):
                    file.write(f"{dppys_selective[pro, per, scen, cust]} ")
    file.write("] \n \n")

    # Write distance matrix between candidate and supplier
    file.write("! Distance matrix between candidate locations and suppliers\nDistanceCandidateSupplier: [")
    for can in range(len(candidate_id_selective[0])):
        file.write(f"\n({can+1} 1) ")
        for supp in range(53):
            file.write(f"{agg_dist_can_supp[can, supp]} ")
    file.write("]\n \n")

    # Write distance matrix between customer and candidate
    file.write("! Distance matrix between candidate locations and customers\nDistanceCandidateCustomer: [")
    for can in range(len(candidate_id_og)):
        file.write(f"\n({can+1} 1) ")
        for cust in range(len(customer_id_selective[0])):
            file.write(f"{agg_dist_customer_only[can, cust]} ")
    file.write("]")

In [24]:
np.array(setup_costs_selective).sum()

54007000